In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

In [3]:
from fastembed import TextEmbedding

embedding_model = TextEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [4]:
query = "What does the BNS 2023 say about arrests?"
query_vector = list(embedding_model.embed([query]))[0]

In [5]:
results = client.query_points(
    collection_name="source-of-truth",
    query=query_vector,
    limit=5,
    with_payload=True
).points

context = "\n\n".join(
    r.payload["text"] for r in results
)

In [8]:
from openai import OpenAI

llm = OpenAI(
    base_url="http://127.0.0.1:1234/v1",
    api_key="lm-studio"  # dummy key
)

response = llm.chat.completions.create(
    model="meta-llama-3.1-8b-instruct",
    messages=[
        {
            "role": "system",
            "content": "Answer strictly using the provided context."
        },
        {
            "role": "user",
            "content": f"""
Context:
{context}

Question:
{query}
"""
        }
    ],
    temperature=0.2
)

print(response.choices[0].message.content)

The Bharatiya Nagarik Suraksha Sanhita (BNS), 2023, specifically mentions that whoever fails to appear at the specified place and time as required by a proclamation published under Section 84 of the BNS, shall be punished with imprisonment for a term which may extend to three years.


In [9]:
for r in results:
    print(r.payload["doc_id"], r.score)

prevofcorruption.pdf 0.68043387
prevofcorruption.pdf 0.652771
prevofcorruption.pdf 0.64211667
bns2023.pdf 0.64040214
bns2023.pdf 0.6344971
